<a href="https://colab.research.google.com/github/maverick98/Group4Capstone/blob/main/decoder_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import necessary libraries

In [1]:
import numpy as np
import string
import tensorflow as tf
from tensorflow import math, matmul, reshape, shape, transpose, cast, float32
from tensorflow.data import Dataset
from tensorflow.keras.layers import Embedding,Dense, Layer,TextVectorization
from tensorflow.keras.backend import softmax
from pickle import load, dump, HIGHEST_PROTOCOL
from sklearn.utils import shuffle
from numpy import savetxt
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow import convert_to_tensor, int64
from tensorflow.keras.layers import LayerNormalization,   ReLU, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers.schedules import LearningRateSchedule
from tensorflow.keras.metrics import Mean
from tensorflow import data, train,  reduce_sum,  equal, argmax,GradientTape, function
from tensorflow.keras.losses import sparse_categorical_crossentropy
from time import time
from pickle import dump
from tensorflow import    linalg, ones, maximum, newaxis
from tensorflow.keras import Model
from pickle import load
from tensorflow import Module
from tensorflow import   TensorArray, argmax,  transpose
from matplotlib.pylab import plt
from numpy import arange

from keras.utils import to_categorical
import numpy as np
from numpy import array
import pandas as pd
import cv2
from glob import glob
import PIL
import time
from tqdm import tqdm
import os
#import gensim
from sklearn.model_selection import train_test_split
from nltk.translate.bleu_score import sentence_bleu
from PIL import Image
from numpy import random
from importlib.machinery import SourceFileLoader
from os.path import join
from google.colab import drive

# Check if connected to GPU

In [2]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Not connected to a GPU


# Mount Google Drive

In [3]:


ROOT = "/content/drive"
drive.mount(ROOT,force_remount=True)




Mounted at /content/drive


In [4]:
PROJ = "MyDrive/Capstone/src" 
PROJECT_PATH = join(ROOT, PROJ)
print("PROJECT_PATH from your Google Drive is ",PROJECT_PATH)
!rm -rf "{PROJECT_PATH}"
!mkdir  "{PROJECT_PATH}"

PROJECT_PATH from your Google Drive is  /content/drive/MyDrive/Capstone/src


# Clone the code from https://sourceforge.net/projects/group4capstone/

In [5]:
%cd "{PROJECT_PATH}"
!pwd
MY_USER_NAME = 'msahu98' # This is your sourceforge.net username
!git clone https://{MY_USER_NAME}@git.code.sf.net/p/group4capstone/code group4capstone-code

/content/drive/MyDrive/Capstone/src
/content/drive/MyDrive/Capstone/src
Cloning into 'group4capstone-code'...
remote: Enumerating objects: 50, done.
remote: Counting objects: 100% (50/50), done.
remote: Compressing objects: 100% (39/39), done.
remote: Total 50 (delta 25), reused 0 (delta 0)
Unpacking objects: 100% (50/50), 6.35 KiB | 8.00 KiB/s, done.


# Load Modules

In [6]:
PROJECT_PATH 
#/content/drive/MyDrive/Capstone/src/group4capstone-code/src/positional_encoding.py

'/content/drive/MyDrive/Capstone/src'

In [7]:
def load_module(module_name):
    module_py=module_name+'.py'
    SourceFileLoader(module_name, join(join(PROJECT_PATH,'group4capstone-code/src'), module_py)).load_module()



In [10]:
load_module('positional_encoding')
load_module('multihead_attention')
load_module('add_normalization')
load_module('feedforward')
load_module('decoder')

# Testing Decoder

In [11]:
from decoder import Decoder

In [14]:
dec_vocab_size=20
input_sequence_length=5
h=8
d_k=64
d_v=64
d_ff=2048
d_model=512
num_layers=6

batch_size=64
dropout_rate=0.1

input_seq=random.random((batch_size,input_sequence_length))
enc_output=random.random((batch_size,input_sequence_length,d_model))

decoder=Decoder(dec_vocab_size,input_sequence_length,h,d_k,d_v,d_model,d_ff,num_layers,dropout_rate)

print(decoder(input_seq, enc_output,None,True))


tf.Tensor(
[[[ 0.13178475 -1.7642318   1.0571268  ...  0.8291508  -1.1990172
   -0.50617254]
  [ 0.18242565 -1.8032032   1.1028193  ...  0.84894186 -1.1993027
   -0.4923751 ]
  [ 0.19774483 -1.8794407   1.1253523  ...  0.8565719  -1.1833655
   -0.47125438]
  [ 0.1580991  -1.9222977   1.1086171  ...  0.83932555 -1.160672
   -0.44662338]
  [ 0.09598697 -1.8996563   1.079998   ...  0.7948668  -1.1515318
   -0.4362285 ]]

 [[ 0.13178475 -1.7642318   1.0571268  ...  0.8291508  -1.1990172
   -0.50617254]
  [ 0.18242565 -1.8032032   1.1028193  ...  0.84894186 -1.1993027
   -0.4923751 ]
  [ 0.19774483 -1.8794407   1.1253523  ...  0.8565719  -1.1833655
   -0.47125438]
  [ 0.1580991  -1.9222977   1.1086171  ...  0.83932555 -1.160672
   -0.44662338]
  [ 0.09598697 -1.8996563   1.079998   ...  0.7948668  -1.1515318
   -0.4362285 ]]

 [[ 0.13178475 -1.7642318   1.0571268  ...  0.8291508  -1.1990172
   -0.50617254]
  [ 0.18242565 -1.8032032   1.1028193  ...  0.84894186 -1.1993027
   -0.4923751 ]
  [